# **🧠 O que é o ciclo BDI?**

> ⚠️ Este Colab não foi feito para execução do codigo, criei por motivos de explicabilidade do codigo. O codigo deve ser rodado localmente no caminho /code.

O **BDI (Belief-Desire-Intention)** é uma arquitetura de agentes baseada na forma como humanos tomam decisões racionais.

O ciclo BDI funciona em três camadas:

*   **Beliefs (Crenças): o que o agente sabe sobre o mundo.**
*   **Desires (Desejos): o que ele quer alcançar.**
*   **Intentions (Intenções): os planos escolhidos para agir com base nas crenças e desejos.**


O agente repete continuamente esse ciclo:

1. **Atualiza suas crenças com novas informações (ex: uma erva daninha detectada).**

2. **Seleciona desejos relevantes com base nesse estado.**

3. **Escolhe planos válidos (cujos contextos estão de acordo com as crenças).**

4. **Executa ações para alcançar os objetivos.**

5. **Volta ao início e reavalia tudo.**








In [ ]:
import requests
import threading
import time
from flask import Flask, request, jsonify

### **Agente de Campo**  —  `agent_field.py`
- Percorre a plantação em tempo real (simulado).
- Fornece streaming de imagens via endpoint `/stream`.
- Recebe notificações de detecção via `/agent` (POST).
- Reage com planos como `parar_monitoramento` e `enviar_coordenadas`.

> A ***PlanLibrary*** armazena os **planos do agente**, associando cada **objetivo (goal)** a um **contexto necessário (pré-condições)** e a uma **sequência de açõe**s que devem ser executadas quando aquele objetivo for ativado.
A função ***get_plan()*** verifica se o contexto de um plano é compatível com **as crenças atuais (beliefs)**. Se for, retorna o plano associado; caso contrário, ignora.

In [ ]:
class PlanLibrary:
    def __init__(self):
        self.plans = {}

    def set_plan_library(self, planlb):
        self.plans = planlb

    def add_plan(self, goal, prec, plan):
        self.plans[goal] = {'context': prec, 'plan': plan}

    def get_plan(self, goal, bb):
        if goal in self.plans:
            if set(self.plans[goal]['context'].items()).issubset(bb.items()):
                return self.plans[goal]['plan']
        return None

### 📦 `class Action`

Essa classe contém as **ações** que o agente pode executar. Cada método representa uma **ação física ou comunicacional**, e é chamado automaticamente quando um plano é ativado.

- **`percorrer_local_definido()`**  
  Simula o agente de campo patrulhando a área.  
  Reseta a crença `envio_coordenadas` para `False`, preparando para futuras atualizações.

- **`parar_monitoramento()`**  
  Quando o agente supervisor detecta anomalias na transmissão do agente, ele para sua atividades e espera no local.

- **`enviar_coordenadas_ao_supervisor()`**  
  Como o supervisor detectou anomalias na plantação, o agente de campo precisa passar suas coordenadas para o agente supervisor mandar auxilio.

In [ ]:
class Action:
    def percorrer_local_definido(self):
        agente.add_beliefs({"envio_coordenadas": False,})
        print("🚜 Percorrendo local definido...")

    def parar_monitoramento(self):
        print("⛔ Monitoramento interrompido.")

    def enviar_coordenadas_ao_supervisor(self):
        print("⚠️ Enviando coordenadas ao supervisor e atualizando minha crença local...")

        # Envia as coordenadas e crença para o supervisor
        try:
            crença_supervisor = {"coordenadas_agente_campo": True}
            res = requests.post("http://127.0.0.1:5003/agent", json=crença_supervisor)
            print(f"📨 Supervisor notificado | Status: {res.status_code}")
        except Exception as e:
            print(f"❌ Falha ao notificar supervisor: {e}")

        agente.add_beliefs({"envio_coordenadas": True,})
        print("📡 Coordenadas enviadas ao supervisor!")

    def aguardar_contenção_de_pragas(self):
        print("⏳ Aguardando contenção de pragas...")



### **🤖 Classe Agent (BDI)**

A classe `Agent` implementa a lógica principal de um agente inteligente baseado no modelo BDI (*Beliefs-Desires-Intentions*).

In [ ]:
class Agent:
    def __init__(self):
        self.beliefs = {}
        self.desires = []
        self.intention = []
        self.plan_library = PlanLibrary()

    def get_desires(self):
        return self.desires.pop() if self.desires else None

    def add_desires(self, desire):
        self.desires.append(desire)

    def add_beliefs(self, beliefs):
        self.beliefs.update(beliefs)

    def set_plan_library(self, pl):
        self.plan_library.set_plan_library(pl)

    def update_intention(self, goal):
        plan = self.plan_library.get_plan(goal, self.beliefs)
        if plan:
            self.intention.extend(plan)

    def execute_intention(self):
        while self.intention:
            next_goal = self.intention.pop()
            if self.plan_library.get_plan(next_goal, self.beliefs) is None:
                action = getattr(Action(), next_goal)
                action()
            else:
                self.intention.extend(self.plan_library.get_plan(next_goal, self.beliefs))


### 🗺️ **Planos Definidos (set_plan_library)**
- **monitorar_trigo:** quando não há erva daninha, o agente patrulha o local.

- **alerta_ervaDaninha:** caso o supervisor ache as ervas daninhas, o agente de campo interrompe a patrulha e envia localização ao supervisor.

- **aguardando_contençãoPragas:** após notificação, aguarda o herbicida.

🌐 Endpoints REST
- **GET /agent:** retorna as crenças atuais do agente.

- **POST /agent:** atualiza crenças com um JSON enviado por outro agente (ex: supervisor).

- **GET /stream:** simula um frame de imagem enviado pelo ESP32CAM.

In [ ]:
app = Flask(__name__)
agente = Agent()

# Crenças iniciais
agente.add_beliefs({
    "erva_daninha": False,
    "local": "ponto_1",
    "envio_coordenadas": False
})

# Biblioteca de planos
agente.set_plan_library({
    "monitorar_trigo": {
        "context": {"erva_daninha": False},
        "plan": ["percorrer_local_definido"]
    },
    "alerta_ervaDaninha": {
        "context": {"erva_daninha": True, "envio_coordenadas": False},
        "plan": ["parar_monitoramento", "enviar_coordenadas_ao_supervisor"]
    },
    "aguardando_contençãoPragas": {
        "context": {"erva_daninha": True, "envio_coordenadas": True},
        "plan": ["aguardar_contenção_de_pragas"]
    }
})


@app.route("/agent", methods=["GET", "POST"])
def agent_endpoint():
    if request.method == "GET":
        return jsonify(agente.beliefs)
    elif request.method == "POST":
        new_beliefs = request.json
        agente.add_beliefs(new_beliefs)
        return jsonify({"msg": "Crenças atualizadas", "novas_crencas": agente.beliefs})

# Simulação do /stream
@app.route("/stream", methods=["GET"])
def stream():
    return jsonify({"frame": "frame_213.jpg"})


### **🔁 Ciclo BDI Contínuo (ciclo_agente)**

Este é o **loop principal** que mantém o agente ativo e deliberando de forma autônoma.

In [ ]:
# Ciclo BDI contínuo
def ciclo_agente():
    while True:
        if not agente.desires:
            agente.add_desires("monitorar_trigo")
            agente.add_desires("alerta_ervaDaninha")
            agente.add_desires("aguardando_contençãoPragas")
        goal = agente.get_desires()
        if goal:
            agente.update_intention(goal)
            agente.execute_intention()
        time.sleep(2)

if __name__ == "__main__":
    threading.Thread(target=ciclo_agente, daemon=True).start()
    app.run(host="0.0.0.0", port=5002)